In [ ]:
#| hide
from base_rbt.base_model import *
from base_rbt.base_lf import *

# base_rbt

> Base functions and classes we use for our hacking on BT / RBT and related ideas

In [ ]:
seed_everything(seed=42)

This file will become your README and also the index of your documentation.

## Install

```sh
!pip install git+https://github.com/hamish-haggerty/base_rbt.git#egg='base_rbt'
```

## How to use

First we need to import some libraries:


In [ ]:
#| hide
import self_supervised
import torch
from fastai.vision.all import *
from self_supervised.augmentations import *
from self_supervised.layers import *

In [ ]:
#| hide
device='cuda' if torch.cuda.is_available() else 'cpu'

Here is a (silly!) modification to the BT loss function. We are just scaling the bt loss function by $0.01$. However, this illustrates the general API if we want to modify the loss function. 

In [ ]:
#| hide
@patch
def lf(self:BarlowTwins, pred,*yb): return 0.01*lf_bt(pred, self.I,self.lmb)

In [ ]:
#| hide
#End to end example
#Get some MNIST data and plonk it into a dls
path = untar_data(URLs.MNIST)
items = get_image_files(path/'training') #i.e. NOT testing!!!
items = items[0:10]
split = RandomSplitter(valid_pct=0.0)
tds = Datasets(items, [PILImageBW.create, [parent_label, Categorize()]], splits=split(items))
dls = tds.dataloaders(bs=2,num_workers=0, after_item=[ToTensor(), IntToFloatTensor()], device=device)

#Full usage of above
ps=500
hs=500
fastai_encoder = create_fastai_encoder(xresnet18(),pretrained=False,n_in=1)
model = create_barlow_twins_model(fastai_encoder, hidden_size=hs,projection_size=ps)# projection_size=1024)
aug_pipelines = get_barlow_twins_aug_pipelines(size=28, rotate=True,flip_p=0,resize_scale=(0.7,1), jitter=False, bw=False,blur=True,blur_p=0.5,blur_s=8, stats=None, cuda=True)
learn = Learner(dls,model, cbs=[BarlowTwins(aug_pipelines, print_augs=True)])
learn.fit(1)